In [ ]:
# Импорт библиотек
import os
import time
import KU_TU
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)

# Состав газов в номинале
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = (
    gas_streams0.at["GTU-PEVD", "N2"],
    gas_streams0.at["GTU-PEVD", "O2"],
    gas_streams0.at["GTU-PEVD", "CO2"],
    gas_streams0.at["GTU-PEVD", "H2O"],
    gas_streams0.at["GTU-PEVD", "Ar"],
)

gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies


####################################################################################################
# Расчет ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, GTU_input, "GTU-KU")
Gas_turbine_res = Gas_turbine.calc()
electric.at["GTU", "N"] = Gas_turbine_res["N"]
electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
electric.at["DK", "N"] = Gas_turbine_res["Ndk"]
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
gas_streams.at["GTU-KU", "P"] = 0.1
gas_streams.at['GTU-KU', 'H'] = gas0.p_t(
    gas_streams.at['GTU-KU', 'P'], gas_streams.at['GTU-KU', 'T'])['h']
Gas_turbine_composition = pd.read_excel(
    "input.xlsx", sheet_name="Gas_composition0", index_col=0)

# Параметры газа на выходе в КУ
gas_streams.loc["GTU-PEVD", "T":"P"] = gas_streams.loc["GTU-KU", "T":"P"]
gas_streams.at["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
gas_streams.at["GTU-PEVD",
               "N2":"Ar"] = Gas_turbine_composition.loc["Fraction", "N2":"Ar"]

# Состав газов при частичной нагрузке
fractiongas = (
    gas_streams.at["GTU-PEVD", "N2"],
    gas_streams.at["GTU-PEVD", "O2"],
    gas_streams.at["GTU-PEVD", "CO2"],
    gas_streams.at["GTU-PEVD", "H2O"],
    gas_streams.at["GTU-PEVD", "Ar"],
)
gas1 = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)

# Основные константы
Calcmethod = "hybr"
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.9

Calctolerance = 10**-2
Teplo = 0
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 30

#Claas KU+TU
KU_and_TU=KU_TU.ku_tu(gas0, gas1,  water, gas_streams0, gas_streams, water_streams0, water_streams, heaters, electric,Calcmethod, KPD_SP, KPD_KN, KPD_to, KPD_PN)

start_time = time.time()

KU_and_TU.calculate(Teplo,Calctolerance,Maxiterations_KU_TU,Maxiterations_cotel,Maxiterations_turbine)


print("Степень сухости пара в ЭВД: ", water.p_h(
    water_streams.at["EVD-IVD", "P"], water_streams.at["EVD-IVD", "H"])["Q"],)
print("Степень сухости пара в ГПК: ", water.p_h(
    water_streams.at["GPK-IND", "P"], water_streams.at["GPK-IND", "H"])["Q"],)

print(f"fin КУ и ТУ:--- {round((time.time() - start_time), 1)} сек. ---")

Pnd_1 0.6691864455899998
Pvd_1 8.209238818333999
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций контура низкого давления


In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# plt.style.use('_mpl-gallery')
# plt.figure(figsize=(20,5))
# plot
fig, ax = plt.subplots()
x = range(0, i+1)
y = Pvd_it
plt.plot(x, y, linewidth=2.0)
fig.set_size_inches(20, 5)

# ax.set(xlim=(0, 8),
#        ylim=(0, 8))
# loc = matplotlib.ticker.LinearLocator(numticks = len(x)-1)
# plt.gca().xaxis.set_major_locator(loc)
ax.set_xticks(x)
plt.show()

In [ ]:
# Вывод газовых потоков
gas_streams

In [ ]:
# Вывод водяных потоков
water_streams

In [ ]:
Errors_water = abs(water_streams - water_streams0)/water_streams0*100
Errors_gas = abs(gas_streams-gas_streams0)/gas_streams0*100
Errors_gas
Errors_water

In [ ]:
gas_streams0

In [ ]:
water_streams0

In [ ]:
electric

In [ ]:
heaters

In [ ]:
GTU_input

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
Qg = [0,
      heaters.loc['PEVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg']+heaters.loc['PPND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD',
                                                                     'Qg']+heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD', 'Qg'] +
      heaters.loc['PPND', 'Qg'] +
      heaters.loc['IND', 'Qg']+heaters.loc['GPK', 'Qg']
      ]
Qw = [0,
      heaters.loc['PEVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg']+heaters.loc['PPND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD',
                                                                     'Qg']+heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD',
                                                                     'Qg']+heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD', 'Qg'] +
      heaters.loc['PPND', 'Qg'] +
      heaters.loc['IND', 'Qg']+heaters.loc['GPK', 'Qg']
      ]
Tg = gas_streams.loc['GTU-PEVD':'GPK-out', 'T']
Tw = [water_streams.loc['PEVD-DROSVD', 'T'],
      water_streams.loc['IVD-PEVD', 'T'],
      water_streams.loc['IVD-PEVD', 'T'],
      water_streams.loc['EVD-IVD', 'T'],
      water_streams.loc['PEN-EVD', 'T'],
      water_streams.loc['PPND-DROSND', 'T'],
      water_streams.loc['IND-PPND', 'T'],
      water_streams.loc['IND-PPND', 'T'],
      water_streams.loc['GPK-REC', 'T'],
      water_streams.loc['REC-GPK', 'T'],
      ]
plt.plot(Qg, Tg, color='red')
plt.plot(Qw[0:2], Tw[0:2], color='blue')
plt.plot(Qw[1:3], Tw[1:3], color='blue')
plt.plot(Qw[3:5], Tw[3:5], color='blue')
plt.plot(Qw[5:7], Tw[5:7], color='blue')
plt.plot(Qw[6:8], Tw[6:8], color='blue')
plt.plot(Qw[8:10], Tw[8:10], color='blue')
plt.xlabel('Q')
plt.ylabel('T')
plt.legend(['gas', 'water'])
plt.show()

In [ ]:
# Импорт библиотек
import os
import time
import cotel
import Turboustanovka
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)

# Состав газов в номинале
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = (
    gas_streams0.at["GTU-PEVD", "N2"],
    gas_streams0.at["GTU-PEVD", "O2"],
    gas_streams0.at["GTU-PEVD", "CO2"],
    gas_streams0.at["GTU-PEVD", "H2O"],
    gas_streams0.at["GTU-PEVD", "Ar"],
)

gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies


####################################################################################################
# Расчет ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, GTU_input, "GTU-KU")
Gas_turbine_res = Gas_turbine.calc()
electric.at["GTU", "N"] = Gas_turbine_res["N"]
electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
electric.at["DK", "N"] = Gas_turbine_res["Ndk"]
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
gas_streams.at["GTU-KU", "P"] = 0.1
gas_streams.at['GTU-KU', 'H'] = gas0.p_t(
    gas_streams.at['GTU-KU', 'P'], gas_streams.at['GTU-KU', 'T'])['h']
Gas_turbine_composition = pd.read_excel(
    "input.xlsx", sheet_name="Gas_composition0", index_col=0)

# Параметры газа на выходе в КУ
gas_streams.loc["GTU-PEVD", "T":"P"] = gas_streams.loc["GTU-KU", "T":"P"]
gas_streams.at["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
gas_streams.at["GTU-PEVD",
               "N2":"Ar"] = Gas_turbine_composition.loc["Fraction", "N2":"Ar"]

# Состав газов при частичной нагрузке
fractiongas = (
    gas_streams.at["GTU-PEVD", "N2"],
    gas_streams.at["GTU-PEVD", "O2"],
    gas_streams.at["GTU-PEVD", "CO2"],
    gas_streams.at["GTU-PEVD", "H2O"],
    gas_streams.at["GTU-PEVD", "Ar"],
)
gas1 = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)

# Основные константы
Calcmethod = "hybr"
KPDnasos = 0.8074
KPD = 0.99
Calctolerance = 10**-2
Teplo = 1
Maxiterations_cotel = 10
Maxiterations_KU_TU = 20
Maxiterations_turbine = 30

####################################################################################################
# Инициализируем все
TU = Turboustanovka.turboustanovka(
    water, water_streams0, water_streams, heaters, electric, KPD_SP=0.99, KPDnasos=0.8)
Whole_cotel = cotel.cotel_all(KPD, KPDnasos,  gas0, gas1, water, Calcmethod,
                              gas_streams0, water_streams0, gas_streams, water_streams, heaters, electric)


Vvd0 = 1/water.p_h(water_streams0.at["PEVD-DROSVD", 'P'],
                   water_streams0.at["PEVD-DROSVD", 'H'])['rho']
Vnd0 = 1/water.p_h(water_streams0.at["PPND-DROSND", 'P'],
                   water_streams0.at["PPND-DROSND", 'H'])['rho']
Gvd0 = water_streams0.at["PEVD-DROSVD", 'G']
Gnd0 = water_streams0.at["PPND-DROSND", 'G']
dPvd0 = water_streams0.at["PEVD-DROSVD", 'P'] - \
    water_streams0.at["DROSVD-TURBVD", 'P']
dPnd0 = water_streams0.at["PPND-DROSND", 'P'] - \
    water_streams0.at["DROSND-TURBND", 'P']

####################################################################################################
# Расчет всего
Pnd_it = []
Pvd_it = []
start_time = time.time()
Max_error_P = 2
Max_error_G = 2
Max_error = 2
Error_nd_P = 2
Error_vd_P = 2
Calctolerance_new = 10**-1
Teplo_overflow = 0

#Первое приближение по давлению
G_gas1 = gas_streams.at["GTU-PEVD", "G"]
G_gas0 = gas_streams0.at["GTU-PEVD", "G"]
g_gas = G_gas1/G_gas0
Pvd_1 = 0.4538+7.7601*g_gas
Pnd_1 = -0.0189+0.6885*g_gas
water_streams.at["PPND-DROSND", 'P'] = Pnd_1
water_streams.at["PEVD-DROSVD", 'P'] = Pvd_1
print('Pnd_1', Pnd_1)
print('Pvd_1', Pvd_1)

for i in range(Maxiterations_KU_TU):

    for j in range(Maxiterations_cotel):

        # Расчет котла
        Whole_cotel.calc(Calctolerance_new, Maxiterations_cotel)
        # print('КУ расчитан')

        Gnd1 = water_streams.at["PPND-DROSND", "G"]
        Gnd2 = water_streams.at["DROSND-TURBND", "G"]
        Gvd1 = water_streams.at["PEVD-DROSVD", "G"]
        Gvd2 = water_streams.at["DROSVD-TURBVD", "G"]

        # Перекидываем расходы
        water_streams.at["DROSVD-TURBVD", 'G'] = Gvd1
        water_streams.at["DROSND-TURBND", 'G'] = Gnd1

        # Перекидываем энтальпии
        water_streams.at["DROSVD-TURBVD",
                         'H'] = water_streams.at["PEVD-DROSVD", 'H']
        water_streams.at["DROSND-TURBND",
                         'H'] = water_streams.at["PPND-DROSND", 'H']
        if i > 0 and Teplo == 1:
            teplofikacia = 1
        else:
            teplofikacia = 0
        # Расчет турбины
        TU_res = TU.calculate(
            teplofikacia, calcmethod="hybr", calctolerance=Calctolerance_new, maxiterations=Maxiterations_turbine
        )
        # print('ТУ рассчитана')

        # основные потоки, которые надо сводить по расходу
        # Calctolerance_new=Calctolerance
        # if Max_error > 0.1 and Max_error <= 1:
        #     print('Max_error', 0.01)
        #     Calctolerance_new = Calctolerance*10
        # if Max_error > 1:
        #     print('Max_error', 1)
        Calctolerance_new = Calctolerance*10
        if  i>2:
            print('Переход к оригинально точности расчета',Calctolerance)
            Calctolerance_new = Calctolerance
        # if Max_error_P < Calctolerance:
        #     print('Max_error_10', 0.01)
        #     Calctolerance_new = Calctolerance
        # if Max_error < 0.01:
        #     print('Max_error', 'else')
        #     Calctolerance_new = Calctolerance

        Error_water_G = abs((water_streams.at["SMESHOD-REC", "G"] -
                             water_streams.at["GPK-IND", "G"])/water_streams.at["GPK-IND", "G"]*100)
        Error_nd_G = abs((Gnd1 - Gnd2)/Gnd1*100)
        Error_vd_G = abs((Gvd1 - Gvd2)/Gvd1*100)
        Max_error_G = max(Error_water_G, Error_nd_G, Error_vd_G)
        Max_error = max(Error_water_G, Error_nd_G,
                        Error_vd_G, Error_nd_P, Error_vd_P)
        print('Max_error_G', Max_error_G)
        print('Error_nd_G', Error_nd_G)
        print('Error_vd_G', Error_vd_G)
        print('Error_water_G', Error_water_G)
        if Error_water_G > 20:
            Teplo_overflow = 1
        if abs(Max_error_G) < Calctolerance_new:
            print(
                "Максимальная погрешность определения расхода в КУ+ПТУ", Max_error_G)
            break
        if j == Maxiterations_cotel - 1:
            print("Достигнуто максимальное количество итераций расхода КУ+ПТУ")

    # Переписываю давления
    P_turb_vd = water_streams.at["DROSVD-TURBVD", 'P']
    P_turb_nd = water_streams.at["DROSND-TURBND", 'P']
    Vvd1 = 1/water.p_h(water_streams0.at["PEVD-DROSVD", 'P'],
                       water_streams0.at["PEVD-DROSVD", 'H'])['rho']
    Vnd1 = 1/water.p_h(water_streams0.at["PPND-DROSND", 'P'],
                       water_streams0.at["PPND-DROSND", 'H'])['rho']
    dPvd = dPvd0 * (Gvd1*Vvd1/Gvd0/Vvd0)**2*Vvd0/Vvd1
    dPnd = dPnd0*(Gnd1*Vnd1/Gnd0/Vnd0)**2*Vnd0/Vnd1
    P_kotel_vd = water_streams.at["PEVD-DROSVD", 'P']
    P_kotel_nd = water_streams.at["PPND-DROSND", 'P']
    P_kotel_nd_new = P_turb_nd+dPnd
    P_kotel_vd_new = P_turb_vd+dPvd
    water_streams.at["PPND-DROSND", 'P'] = (P_kotel_nd_new+P_kotel_nd)/2
    water_streams.at["PEVD-DROSVD", 'P'] = (P_kotel_vd_new+P_kotel_vd)/2

    # Закидываю давления в массив
    Pnd_it.append(round(water_streams.loc['PPND-DROSND', 'P'], 5))
    Pvd_it.append(round(water_streams.loc['PEVD-DROSVD', 'P'], 5))
    print('Pnd_it', Pnd_it)
    print('Pvd_it', Pvd_it)

    # Ошибки расчета
    Error_nd_P = abs((P_kotel_nd - P_kotel_nd_new)/P_kotel_nd*100)
    Error_vd_P = abs((P_kotel_vd - P_kotel_vd_new)/P_kotel_vd*100)
    Max_error = max(Error_water_G, Error_nd_G,
                    Error_vd_G, Error_nd_P, Error_vd_P)
    print('Error_nd_P', Error_nd_P)
    print('Error_vd_P', Error_vd_P)
    Max_error_P = max(Error_nd_P, Error_vd_P)
    print('Max_error', Max_error)
    print(
        f"Время {i+1} итерации расчета КУ+ТУ:--- %s сек. --- {round((time.time() - start_time), 1)}")
    if Teplo_overflow == 1:
        print('Слишком большая теплофикационная мощность, расчет окончен.')
        print('Для правильного расчета необходимо повысить мощность ГТУ или уменьшить мощность теплофикации.')
    if abs(Max_error) < Calctolerance and Calctolerance_new == Calctolerance:
        print(
            "Максимальная погрешность определения расходов", Error_water_G)
        break
    if i == Maxiterations_KU_TU - 1:
        print("Достигнуто максимальное количество итераций давления КУ+ПТУ", i+1)

print("Степень сухости пара в ЭВД: ", water.p_h(
    water_streams.at["EVD-IVD", "P"], water_streams.at["EVD-IVD", "H"])["Q"],)
print("Степень сухости пара в ГПК: ", water.p_h(
    water_streams.at["GPK-IND", "P"], water_streams.at["GPK-IND", "H"])["Q"],)

print(f"fin КУ и ТУ:--- {round((time.time() - start_time), 1)} сек. ---")